This is the project I am developing for my masters. Differently from the QNN equivariant to the Z2 x Z2 group task, this one proposes the idea of an **Equivariant Quantum Convolutional Neural Network!**

The original CNN/QCNN already is equivariant to the group of **discrete translations**. In our case, I extend this to the group of **discrete rotations** too. This notebook still has improvements to be made, like testing it on some dataset, but we can already see that equivariance is achieved by all qubit permutations in the Parametrized Quantum Circuit.

In [ ]:
!pip install pennylane

# Define our group representations (discrete rotations)

In [8]:
I = torch.Tensor([[1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]])


rot_90 = torch.Tensor([[0, 1, 0, 0],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1],
                    [1, 0, 0, 0]])


rot_180 = torch.Tensor([[0, 0, 1, 0],
                    [0, 0, 0, 1],
                    [1, 0, 0, 0],
                    [0, 1, 0, 0]])


rot_270 = torch.Tensor([[0, 0, 0, 1],
                        [1, 0, 0, 0],
                        [0, 1, 0, 0],
                        [0, 0, 1, 0]])

In [20]:
import pennylane as qml
import numpy as np

import time

import torch.nn as nn

from functools import partial

dev = qml.device("default.qubit", wires=4)


def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w, n_qubits, rot=0):
    """Layer of parametrized qubit rotations around the y axis.
    """
    
    if rot == 90:
        w = rot_90.T @ w
    if rot == 180:
        w = rot_180.T @ w
    if rot == 270:
        w = rot_270.T @ w

    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


@qml.qnode(dev, interface="torch")
def quantum_net(q_input_features, q_weights_flat, q_depth, n_qubits, rot=0):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node. This one has always rot = 0
    RY_layer(q_input_features, n_qubits, 0)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        RY_layer(q_weights[k],  n_qubits, rot) # (k - rot/90) % q_depth implements the rotation as simple permutations.

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]

    return tuple(exp_vals)



class EQCNN(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self, q_depth, q_delta, n_qubits, check_equivariance = False, device="cpu"):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()

        self.device = device
        self.n_qubits = n_qubits
        self.q_depth = q_depth
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))

        self.post_net = nn.Linear(196, 10) # 10 classes for MNIST

        # important for final invariance!
        self.projection_layer = partial(torch.mean, dim=(-2,-1))#(-3, -2, -1))

        if check_equivariance:
            self.check_equivariance()

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """
        
        bsz = input_features.shape[0]
        size = 28


        #CHECK RESCALING OVER pi/2
        input_features = input_features.view(bsz, size, size) * np.pi/2


        # this will contain the tensor where the pooling op will be done
        global_out = torch.Tensor(0, 1)#self.n_qubits)
        global_out = global_out.to(self.device)

        for c in range(0, size, 2):
            for r in range(0, size, 2):

                # 1 because original return is like [1.0, 0.2, -0.9, 0.5], after max pool <=> [1.0]
                tmp = torch.Tensor(0, 1)
                elements = torch.transpose(torch.cat((input_features[:, c, r], input_features[:, c, r+1], input_features[:, c+1, r], input_features[:, c+1, r+1])).view(4, bsz), 0, 1)
                for e in elements:

                    # Apply the quantum circuit to each element of the batch and append to q_out
                    q_out = torch.Tensor(0, self.n_qubits)
                    q_out = q_out.to(self.device)
                    for rotation in [0, 90, 180, 270]:
                        q_out_elem = quantum_net(e, self.q_params, self.q_depth, self.n_qubits, rotation).float().unsqueeze(0)
                        q_out = torch.cat((q_out, q_out_elem))

                    # projection layer for obtaining equivariance!!!
                    q_out = self.projection_layer(q_out).unsqueeze(0).unsqueeze(0) #torch.max(self.projection_layer(q_out)).unsqueeze(0).unsqueeze(0)

                    tmp = torch.cat((tmp, q_out))

                if len(global_out) > 0:
                    global_out = torch.cat((global_out, tmp), -1) #check if axis is right, if pooling is being done on right dimension
                else:
                    global_out = torch.cat((global_out, tmp), 0)
                
        # return post processing layer!!!
        global_out = self.post_net(global_out)

        return global_out




    def check_equivariance(self):

        print("[*] Creating random tensor for equivariance testing..")
        rand_tensor = torch.rand(4).unsqueeze(0)
        rand_tensor = torch.cat((rand_tensor, (rot_90.T @ rand_tensor[0]).unsqueeze(0)))
        rand_tensor = torch.cat((rand_tensor, (rot_180.T @ rand_tensor[0]).unsqueeze(0)))
        rand_tensor = torch.cat((rand_tensor, (rot_270.T @ rand_tensor[0]).unsqueeze(0)))
        print("\n[+] New tensor x = ", rand_tensor, "\n")
        
        thetas = ["0  ", "π/2", "π  ", "3π/2"]
        for e, theta in zip(rand_tensor, thetas):
            q_out = torch.Tensor(0, self.n_qubits)
            q_out = q_out.to(self.device)
            for rotation in [0, 90, 180, 270]:
                q_out_elem = quantum_net(e, self.q_params, self.q_depth, self.n_qubits, rotation).float().unsqueeze(0)
                q_out = torch.cat((q_out, q_out_elem))

            # q_out = torch.max(q_out, dim=0).values#.unsqueeze(0)#self.projection_layer(q_out).unsqueeze(0) #torch.max(self.projection_layer(q_out)).unsqueeze(0).unsqueeze(0)

            q_out = self.projection_layer(q_out).unsqueeze(0)
            print("(ℒ_θ(x) | θ = {}) = ".format(theta), q_out)


# Let's check for equivariance!

In [21]:
equivariant_model = EQCNN(q_depth=1, q_delta=0.1, n_qubits=4, check_equivariance=True)

[*] Creating random tensor for equivariance testing..

[+] New tensor x =  tensor([[0.1234, 0.1259, 0.8803, 0.0018],
        [0.0018, 0.1234, 0.1259, 0.8803],
        [0.8803, 0.0018, 0.1234, 0.1259],
        [0.1259, 0.8803, 0.0018, 0.1234]]) 

(ℒ_θ(x) | θ = 0  ) =  tensor([-0.2583], grad_fn=<UnsqueezeBackward0>)
(ℒ_θ(x) | θ = π/2) =  tensor([-0.2583], grad_fn=<UnsqueezeBackward0>)
(ℒ_θ(x) | θ = π  ) =  tensor([-0.2583], grad_fn=<UnsqueezeBackward0>)
(ℒ_θ(x) | θ = 3π/2) =  tensor([-0.2583], grad_fn=<UnsqueezeBackward0>)
